
        select bank_id, number_of_chain, swtc+swtv total, wight
        from(
            select bank_id, number_of_chain, sum(wight) wight , sum(transaction_value) swtv, sum(transaction_cost) swtc
            from(
                select bank_id, number_of_chain, sum(new_count) wight, transaction_cost, transaction_value
                from(
                    select *
                    from(
                        select bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count(count) new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count = 0
                        group by bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1, transaction_cost, transaction_value) as b0
 
                    union all
                    select *
                    from(
                        select bank_id, length(transaction_type_chain) - length(replace(transaction_type_chain, ';', '')) + 1 number_of_chain,  count new_count, transaction_cost, transaction_value
                        from data_dataset
                        where count != 0) as b1) as b2
                group by bank_id, number_of_chain, transaction_cost, transaction_value) as b3
            group by bank_id, number_of_chain) as b4
            where bank_id = {{number_of_bankid}}
            order by total desc

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../../github_source/tranasaction_dataset.csv', parse_dates=['time_internal'])
data = data.replace(np.nan,0)
data['length_chain'] = data['transaction_type_chain'].str.count(';')+1

In [4]:
tmp_data = data[data['count'] == 0]
tmp_data
dg1 = tmp_data.groupby(by=['bank_id', 'length_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'count'}).reset_index()

In [5]:
tmp_data2 = data[data['count'] != 0]
dg2 = tmp_data2[['count', 'bank_id', 'length_chain', 'transaction_cost', 'transaction_value']]

In [6]:
dg = pd.concat([dg1, dg2])

In [7]:
dg = dg.groupby(by=['bank_id', 'length_chain', 'transaction_cost', 'transaction_value']).agg({'count': 'sum'}).reset_index()

In [8]:
dg = dg.groupby(by=['bank_id', 'length_chain']).agg({
    'count' : 'sum',
    'transaction_cost' : 'sum',
    'transaction_value' : 'sum'
}).reset_index()

In [9]:
dg['total_value'] = dg['transaction_cost'] + dg['transaction_value']
dg = dg.drop(['transaction_cost', 'transaction_value'], axis='columns')

In [18]:
your_bank_id = 13335
dg[dg['bank_id'] == your_bank_id].sort_values('total_value', ascending=False)

,bank_id,length_chain,count,total_value
4560,13335,6,287687,32174085508
4561,13335,7,293587,15370500633
4563,13335,9,205181,12814459626
4562,13335,8,194422,9658698797
4564,13335,10,40603,9428213986
4566,13335,12,11414,2010271669
4559,13335,5,36845,1940969550
4565,13335,11,10386,1572919988
4568,13335,14,6680,1434812975
4567,13335,13,5132,972414754
